Let's take a look at `setup.py`, which we use to, well, set up stuff:

In [1]:
import setup as s

Objects of the class `s.Strategies` tell the rest of the library what's the structure of the games that we will be dealing with